In [1]:
import pandas as pd
import re
import numpy as np
import json

In [4]:
reid_df = pd.read_csv('./log_SGD_continue_1e-4/results.csv')
with open('../RT-DETR/submissions/0.88x no aug best conf 0.8 no reid lb 0.306.json', 'r') as f:
    mapping = json.load(f)
batched_mapping = {}
for k, v in mapping.items():
    k = k.replace('\\', '/')
    batched_mapping['/'.join(k.split('/')[:4] + ['test_old'] + k.split('/')[5:])] = v  # 6: for image folder structure mapping json, 5: for new one
mapping = batched_mapping

In [5]:
pattern = re.compile(r'([-\d]+)_c(\d+)')
def combine(row):
    fp = row['file_path'].replace('\\', '/')
    row['Image_ID'] = 'image_' + pattern.search(fp).groups()[-1].zfill(4)
    row['class'] = row['result']
    row['confidence'] = mapping[fp]['confidence']
    row['ymin'] = mapping[fp]['ymin']
    row['xmin'] = mapping[fp]['xmin']
    row['ymax'] = mapping[fp]['ymax']
    row['xmax'] = mapping[fp]['xmax']
    return row

cols = ['Image_ID', 'class', 'confidence', 'ymin', 'xmin', 'ymax', 'xmax']

reid_df_new = reid_df.apply(combine, axis=1)[cols]
reid_df_new.to_csv('submissions/0.88x conf 0.8 no reid lb 0.306_reid_0.7868934651156535_thresh_swinB_RR.csv',index=False)

In [ ]:
mapping